In [1]:
#!git clone https://github.com/n2ak/attentive_translator
%cd attentive_translator

/content/attentive_translator


In [2]:
!pwd

/content/attentive_translator


In [1]:
%load_ext autoreload
%autoreload 2
import torch
from model import *
from train import *

from attentive_translator.test import *
import string
from attentive_translator.utils import *
torch.manual_seed(1337)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'attentive_translator'

In [7]:
src_path = "./resources/news-commentary-v9.fr-en.en"
dst_path = "./resources/news-commentary-v9.fr-en.fr"
N = 3
n_embeddings = 32
n_head = 4
device = "cuda" if torch.cuda.is_available() else 'cpu'
block_size = 50
start_token,end_token = "[","]"

In [13]:
src,src_vocab,dst,dst_vocab,chars = prepare_data(
    src_path,
    dst_path,
    50,
    add_start_end_tokens=True,
    num_lines=100,
    start_token=start_token,
    end_token=end_token,
    remove_lines_containing_tokens=True
)
encoder_vocab_size = len(src_vocab)
decoder_vocab_size = len(dst_vocab)
encoder_vocab_size,decoder_vocab_size,len(chars)

(53, 59, 12740)

In [ ]:
# for input,target,next_char in zip(decode_all(src,src_vocab),decode_all(dst,dst_vocab),decode(chars,dst_vocab)):
#     print(input,"   ",target,"   ---->   ",next_char)

In [ ]:
model = AttentiveTranslator(
    N,
    encoder_vocab_size,
    decoder_vocab_size,
    32,
    4,
    src.shape,
    dst.shape,
    4,
    4,
    device=device
).to(device=device)
optim = torch.optim.Adam(
    model.parameters()
)
load_weights(model,optim,"./save_model.pt")


In [39]:
a = {"ff":3}
a.update({"N":4})
a

{'ff': 3, 'N': 4}

In [17]:
src_train,src_valid,dst_train,dst_valid,chars_train,chars_valid = split(src,dst,chars)
len(src_train),len(src_valid),len(dst_train),len(dst_valid),len(chars_train),len(chars_valid)

(1344076, 336020, 1344076, 336020, 1344076, 336020)

In [18]:
batch_size = 32 * 2

#model = model.to(memory_format=torch.channels_last)
optim = torch.optim.Adam(
    model.parameters()
)
loss_fn = torch.nn.functional.cross_entropy
train_loader = arrays_to_loader(src_train,dst_train,chars_train,batch_size=batch_size)

Loss: 1.8434727191925049: 100%|██████████| 42003/42003 [15:50<00:00, 44.19it/s]


In [ ]:
epochs = 1
train(model,optim,loss_fn,epochs,train_loader,device,dst_vocab)

In [24]:
#from test import translate
translate(
    model,
    "its better to be quiet".lower(),
    block_size,
    src_vocab,
    dst_vocab,
    device,
    start_char=start_token,
    stop_char=end_token,
)

"la prisse de l'autre de l'autre de la prisse de la"